In [2]:
# Import dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd


## NASA Mars News

In [3]:
# URL of page to be scraped
news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'


In [4]:
# Retrieve page with the requests module
response = requests.get(news_url)


In [5]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')


In [6]:
# Examine the results, then determine element that contains sought info
# print(soup.prettify())


In [7]:
# Collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.
news_title = soup.find('div', class_="content_title").find('a').text
print(news_title)

news_p = soup.find('div', class_="rollover_description_inner").text
print(news_p)



NASA Garners 7 Webby Award Nominations


Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.



## JPL Mars Space Images - Featured Image

In [8]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image 
# and assign the url string to a variable called featured_image_url.
!which chromedriver

/usr/local/bin/chromedriver


In [9]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [10]:
# URL to visit and open with Splinter
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
base_url = (jpl_url.split('/spaceimages'))[0]
browser.visit(jpl_url)

In [11]:
# HTML object
jpl_html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(jpl_html, 'html.parser')
# Retrieve the image URL
image = soup.find('article', class_='carousel_item')["style"]
def get_url(str):
    parts = str.split("'") 
    return parts[1]
print(image)


background-image: url('/spaceimages/images/wallpaper/PIA14925-1920x1200.jpg');


In [12]:
# Get unique image URL
featured_image_url = base_url + get_url(image)
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA14925-1920x1200.jpg'

## Mars Weather

In [14]:
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

In [19]:
# HTML object
weather_html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(weather_html, 'html.parser')
# Retrieve text contained in first tweet
tweet = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')

for child in tweet.find_all('a'):
    child.decompose()

print(tweet.get_text())

InSight sol 136 (2019-04-14) low -97.3ºC (-143.2ºF) high -16.5ºC (2.4ºF)
winds from the WNW at 4.3 m/s (9.6 mph) gusting to 10.8 m/s (24.2 mph)
pressure at 7.30 hPa


## Mars Facts

In [14]:
# pass the URL
facts_url = "https://space-facts.com/mars/"

In [15]:
# read the URL using read_html. This will find any table structures in the HTML
# There is only one table on this page, shown below
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [16]:
# pass to a DataFrame
df = tables[0]
df.columns = ['Fact', 'Value']
df.head()

,Fact,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [17]:
# set first column as Index
df.set_index('Fact', inplace=True)
df.head()

,Value
Fact,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"


In [18]:
# convert to HTML table string
html_table = df.to_html()
html_table


'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Fact</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </t

In [19]:
# replace '/n' with ''
html_table.replace('\n', '')


'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Fact</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

## Mars Hemispheres

In [20]:
# Mars Hemispheres URL
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'


In [21]:
browser.visit(hemispheres_url)

In [22]:
html = browser.html

In [23]:
soup = BeautifulSoup(html, 'html.parser')

In [24]:
# Split the base_url
base_hemispheres_url = (hemispheres_url.split('/search'))[0]
base_hemispheres_url

'https://astrogeology.usgs.gov'

In [25]:
# List to hold hemisphere image URLs
hemisphere_image_urls = []


In [26]:
# Find the class to that holds each hemisphere info
hemispheres = soup.find_all('div', class_='description')


In [27]:
# For loop to extract info for each hemisphere
for hemisphere in hemispheres:
    
    # Create dictionary to hold title and URL
    hemisphere_title_url = {}  
    
    # Find the title and remove "Enhanced"
    hemisphere_title = hemisphere.find('h3').text
    hemisphere_title_url['title'] = hemisphere_title.split(' Enhanced')[0]
    
    # Find the html route to each page
    route = hemisphere.find('a', class_='itemLink product-item')['href']
    individual_hemisphere_route = base_hemispheres_url + route
    browser.visit(individual_hemisphere_route)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    hemisphere_image_url = soup.find('div', class_='downloads').find('ul').find('li').find('a')['href']
    
    # Image URL added to dictionary
    hemisphere_title_url['image_url'] = hemisphere_image_url
    
    # Dictionary added to list
    hemisphere_image_urls.append(hemisphere_title_url)
    
    print(hemisphere_title)
    print(individual_hemisphere_route)
        

Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
Valles Marineris Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [28]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]